<a href="https://colab.research.google.com/github/mmmmiikkkka/my-macro-project/blob/main/midterm2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

pwt90 = pd.read_stata('https://www.rug.nl/ggdc/docs/pwt90.dta')

oecd_countries = [
    'Australia', 'Austria', 'Belgium', 'Canada', 'Denmark', 'Finland', 'France', 'Germany', 'Greece',
    'Iceland', 'Ireland', 'Italy', 'Japan', 'Netherlands', 'New Zealand', 'Norway', 'Portugal',
    'Spain', 'Sweden', 'Switzerland', 'United Kingdom', 'United States',
]

data = pwt90[
    pwt90['country'].isin(oecd_countries) &
    pwt90['year'].between(1990, 2019)
]

relevant_cols = ['countrycode', 'country', 'year', 'rgdpna', 'rkna', 'pop', 'emp', 'avh', 'labsh', 'rtfpna']
data = data[relevant_cols].dropna()

# Calculate additional variables
data['alpha'] = 1 - data['labsh']
data['hours'] = data['emp'] * data['avh']  # L
data['y_n'] = data['rgdpna'] / data['pop']  # Y/N
data['y_l'] = data['rgdpna'] / data['hours'] #Y/L
data['k_l'] = data['rkna'] / data['hours'] #K/L
data['cap_term'] = data['rkna'] / data['pop'] # K/Y
data['lab_term'] = data['hours'] / data['pop']  # L/N
data = (
    data.sort_values('year')
        .groupby('countrycode', group_keys=False)
        .apply(lambda x: x.assign(
            alpha=1 - x['labsh'],
            y_n_shifted=100 * x['y_n'] / x['y_n'].iloc[0],
            tfp_term_shifted=100 * x['rtfpna'] / x['rtfpna'].iloc[0],
            cap_term_shifted=100 * x['cap_term'] / x['cap_term'].iloc[0],
            lab_term_shifted=100 * x['lab_term'] / x['lab_term'].iloc[0]
        ))
).reset_index(drop=True).dropna()

#成長会計
def calculate_growth_rates(country_data):

    country_data = country_data.sort_values('year')
    start_year_actual = country_data['year'].min()
    end_year_actual = country_data['year'].max()

    start_data = country_data[country_data['year'] == start_year_actual].iloc[0]
    end_data = country_data[country_data['year'] == end_year_actual].iloc[0]

    years = end_data['year'] - start_data['year']

    #各種成長率
    g_y = (np.log(end_data['y_l']) - np.log(start_data['y_l'])) / years * 100  # Y/L
    g_k = (np.log(end_data['k_l']) - np.log(start_data['k_l'])) / years * 100  # K/L
    g_l = (np.log(end_data['hours']) - np.log(start_data['hours'])) / years * 100  # L

    alpha_avg = country_data['alpha'].mean()
    g_a = g_y - alpha_avg * g_k  #TFP

    #各要素の寄与
    capital_deepening_contrib = alpha_avg * g_k
    labor_deepening_contrib = (1 - alpha_avg) * g_l
    tfp_growth_calculated = g_a

    #GDP成長率＝TFP成長率＋α＊資本成長率＋（1－α）＊労働成長率
    total_growth = tfp_growth_calculated + capital_deepening_contrib + labor_deepening_contrib

    #構成比
    tfp_share = (tfp_growth_calculated / g_y)
    cap_share = (capital_deepening_contrib / g_y)

    return {
        'Country': start_data['country'],
        'Growth Rate': round(g_y, 2),
        'TFP Growth': round(tfp_growth_calculated, 2),
        'Capital Deepening': round(capital_deepening_contrib, 2),
        'TFP Share': round(tfp_share, 2),
        'Capital Share': round(cap_share, 2)
    }

#表の作成
results_list = data.groupby('country').apply(calculate_growth_rates).dropna().tolist()
results_df = pd.DataFrame(results_list)

avg_row_data = {
    'Country': 'Average',
    'Growth Rate': round(results_df['Growth Rate'].mean(), 2),
    'TFP Growth': round(results_df['TFP Growth'].mean(), 2),
    'Capital Deepening': round(results_df['Capital Deepening'].mean(), 2),
    'TFP Share': round(results_df['TFP Share'].mean(), 2),
    'Capital Share': round(results_df['Capital Share'].mean(), 2)
}
results_df = pd.concat([results_df, pd.DataFrame([avg_row_data])], ignore_index=True)

print("Growth Accounting in OECD Countries: 1990-2019 period")
print("="*85)
print(results_df.to_string(index=False))


Growth Accounting in OECD Countries: 1990-2019 period
       Country  Growth Rate  TFP Growth  Capital Deepening  TFP Share  Capital Share
     Australia         1.63        0.92               0.72       0.56           0.44
       Austria         1.46        0.66               0.80       0.45           0.55
       Belgium         0.93        0.29               0.64       0.31           0.69
        Canada         1.17        0.47               0.70       0.40           0.60
       Denmark         1.26        0.63               0.63       0.50           0.50
       Finland         1.74        0.94               0.80       0.54           0.46
        France         1.43        0.62               0.82       0.43           0.57
       Germany         1.72        0.92               0.80       0.54           0.46
        Greece         1.13        0.14               0.99       0.12           0.88
       Iceland         1.65        1.26               0.39       0.76           0.24
       Irel

<ipython-input-2-96f266de75d6>:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.assign(
<ipython-input-2-96f266de75d6>:82: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results_list = data.groupby('country').apply(calculate_growth_rates).dropna().tolist()


・与えられたファイルをそのまま回してもエラーが出るようになっている
・一つ一つに説明をいれなくてはいけない
・経済学として何をしているか理解する

☆GDP成長率＝TFP成長率＋α＊資本成長率＋（1－α）＊労働成長率